In [ ]:
# Versión mínima
archivos <- list.files(pattern = "prediccion_prod_completa_.*\\.txt$")

if(length(archivos) == 0) stop("No hay archivos de predicciones")

# Leer todas las predicciones
lista_preds <- lapply(archivos, fread, sep = "\t")

# Verificar que todas tengan misma estructura
clientes_base <- lista_preds[[1]]$numero_de_cliente
for(i in seq_along(lista_preds)) {
  if(!identical(lista_preds[[i]]$numero_de_cliente, clientes_base)) {
    stop("Inconsistencia en numero_de_cliente del archivo: ", archivos[i])
  }
}

# Calcular promedio
matriz_probs <- sapply(lista_preds, function(x) x$prob)
vprob_promedio <- rowMeans(matriz_probs)

# Crear output ordenado
tb_final <- data.table(numero_de_cliente = clientes_base, prob = vprob_promedio)
setorder(tb_final, -prob)
tb_final[1:11000, Predicted := 1L]
tb_final[is.na(Predicted), Predicted := 0L]

fwrite(tb_final[Predicted == 1L, .(numero_de_cliente, Predicted)], 
       "submission.csv", sep = ",")